## 제품

- page 넘어가는 것 + 4개 상품 리스트 내려가는 것 구현 (주소값이 똑같음)
- 총 리뷰 개수와 평점은 체험단이 제외되지 않음

In [2]:
from selenium import webdriver as wd
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

import time
import pandas as pd
import numpy as np


driver=wd.Chrome()
url="https://www.oliveyoung.co.kr/store/main/main.do?oy=0"
driver.get(url)

In [3]:
#비건 검색
driver.find_element(By.CSS_SELECTOR, '.inp_placeholder').send_keys("비건")
driver.find_element(By.CSS_SELECTOR, '.inp_placeholder').send_keys(Keys.ENTER)

In [10]:
#카테고리 항목 가져오기
b_category=driver.find_elements(By.CSS_SELECTOR, 'form > div > div > div > ul > li > a')
cats=[cat.get_attribute('id') for cat in b_category]

category=[]

category.append(cats[4]) #마스크팩
category.append(cats[6]) #스킨케어

print(category)
print(len(category))

['category_10000010009', 'category_10000010001']
2


In [25]:
#카테고리 +항목 클릭
driver.find_element(By.CLASS_NAME, 'btnMore').click() 

In [28]:
product_tables=[]

for i in category: #카테고리별
    driver.find_element(By.ID, i).click()
    
    for j in range(1,5): #제품 클릭 / range 늘리기
        path = '#w_cate_prd_list > li:nth-child({}) > div > a > img'.format(j)
        a= driver.find_element(By.CSS_SELECTOR, path)
        a.click()

        data_table={} #제품에서 필요한 내용들 가져오기
        data_table['제품명'] = driver.find_element(By.CLASS_NAME, 'prd_name').text
        data_table['브랜드']=driver.find_element(By.CLASS_NAME, 'pd_arrow_link').text
        try:
            data_table['가격']=driver.find_element(By.CLASS_NAME, 'price-1').text #정가 가져오기
        except:
            data_table['가격']=driver.find_element(By.CLASS_NAME, 'price-2').text #가격 할인가(없으면 정가)로 가져옴 
        data_table['카테고리']=driver.find_element(By.CLASS_NAME, 'cate_y').text
        

        driver.find_element(By.CLASS_NAME, 'goods_reputation').click() #리뷰 부분(체험단이 제외되지 않음)
        time.sleep(1)
        count = driver.find_element(By.CSS_SELECTOR, '#gdasContentsArea > div > div.product_rating_area.review-write-delete > div > div.star_area > p.total > em').text
        if "," in count:
            count=count.replace(',','')
        count=count.strip(" ")
        data_table['총 리뷰 수']=int(count)
        data_table['총 평점']=driver.find_element(By.CSS_SELECTOR, "#gdasContentsArea > div > div.product_rating_area.review-write-delete > div > div.star_area > p.num > strong").text


        driver.find_element(By.CLASS_NAME, 'goods_buyinfo').click() #구매정보 부분
        time.sleep(1)
        data_table['용량']=driver.find_element(By.CSS_SELECTOR, '#artcInfo > dl:nth-child(2) > dd').text
        data_table['성분']=driver.find_element(By.CSS_SELECTOR, '#artcInfo > dl:nth-child(8) > dd').text
        data_table['이미지']=driver.find_element(By.ID, 'mainImg').get_attribute('src')


        product_tables.append(data_table)
        driver.back() #뒤로 가기

product_df=pd.DataFrame.from_dict(product_tables)
product_df

,제품명,브랜드,가격,카테고리,총 리뷰 수,총 평점,용량,성분,이미지
0,비건이펙트 클린 앤 글로우 청보리 퍼스트 LHA 토너 패드 60매,비건이펙트,"22,000\n원",마스크팩,191,4.8,60매,"쌀겨수(800,625ppm), 부틸렌글라이콜, 다이프로필렌글라이콜, 글리세린, 다이...",https://image.oliveyoung.co.kr/uploads/images/...
1,아워 비건 어성초 시카 마스크 1매,아워비건,"3,000\n원",마스크팩,197,4.6,25ml,"정제수, 약모밀추출물(어성초추출물, 5%), 부틸렌글라이콜, 글리세린, 병풀추출물,...",https://image.oliveyoung.co.kr/uploads/images/...
2,[앨리스 소희PICK][비건진정대왕패드] 제이준 잇 비건 편백 시카 카밍 패드 50...,제이준,"27,000\n원",마스크팩,75,4.9,[본품] 잇 비건 편백 시카 카밍 패드 50 Pads (250 ml / 8.45 f...,"편백수(65 %), 정제수, 부틸렌글라이콜, 프로판다이올, 1,2-헥산다이올, 트레...",https://image.oliveyoung.co.kr/uploads/images/...
3,[단독기획] 오어스 비건 민들레진정 수분결 토너패드 60매 (+60매 리필증정 기획),오어스,"24,000\n원",마스크팩,170,4.7,160ml 60매 (5.41 fl.oz.) * 2개입,"정제수@글리세린@다이프로필렌글라이콜@프로판다이올@1,2-헥산다이올@서양민들레꽃/잎/...",https://image.oliveyoung.co.kr/uploads/images/...
4,비건이펙트 클린 앤 글로우 LHA 발효 녹두 앰플 50ml,비건이펙트,"24,000\n원",스킨케어,66,4.7,50ml,"락토바실러스/녹두씨발효여과물, 다이프로필렌글라이콜, 부틸렌글라이콜, 1,2-헥산다이...",https://image.oliveyoung.co.kr/uploads/images/...
5,비건이펙트 클린 앤 글로우 수분초 크림 50g,비건이펙트,"24,000\n원",스킨케어,55,4.9,50g,"위버대나무줄기추출물, 글리세린, 프로판다이올, 하이드로제네이티드폴리데센, 1,2-헥...",https://image.oliveyoung.co.kr/uploads/images/...
6,[비건토너] 스킨푸드 미나리 토너 300ml,스킨푸드,"24,000\n원",스킨케어,169,4.8,300ml,"정제수, 미나리추출물(15%), 메틸프로판다이올, 1,2-헥산다이올, 소듐시트레이트...",https://image.oliveyoung.co.kr/uploads/images/...
7,나인위시스 비건 화이트 톤업 크림 50ml 더블기획,나인위시스,"25,800\n원",스킨케어,1477,4.8,50ml * 2개,"해수,정제수,사이클로펜타실록세인,에칠헥실 메톡시신나메이트,사이클로헥사실록세인,페닐 ...",https://image.oliveyoung.co.kr/uploads/images/...


In [29]:
product_df.to_csv("C:/Users/pc/Desktop/민지/동아리/프로젝트/제품.csv", encoding='utf-8-sig')